In [1]:
import pandas as pd
from utils import get_path, group_cols
from datatable import dt, f

# Descrição

Neste notebook iremos formar a tabela deslocamento, com origem e destino.

# Tabelas base

## Tabela Procedimentos

### Colunas da tabela

In [2]:
cols_deslc = {
  'ano': 'ano',
  'parto_normal': 'parto_normal',
  'cod_municipio': 'origem',
  'hosp_cod_municipio': 'destino',
  'regiao_saude': 'regiao_saude',
  'hosp_regiao_saude': 'hosp_regiao_saude',
  'cnes': 'hosp_cnes',
  'count': 'count',
  ...: None,
  # 'idade'
}

### Linhas da tabela

In [3]:
def filter_deslc(df):
  df_filtered = df[
    (
      f.ano == 2010
    ) | (
      f.ano == 2011
    ) | (
      f.ano == 2018
    ) | (
      f.ano == 2019
    ), :]
  return df_filtered

### Carregando tabela

In [4]:
path_deslc = get_path('GESTANTES', 'procedimentos.csv.gzip')
df_deslc = dt.fread(path_deslc, columns=cols_deslc)
df_deslc = filter_deslc(df_deslc)
df_deslc

,ano,parto_normal,hosp_cnes,destino,hosp_regiao_saude,origem,regiao_saude,count
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2010,1,2077388,355030,3501,355030,3501,362
1,2010,1,2077388,355030,3501,355030,3501,333
2,2010,1,2077388,355030,3501,355030,3501,331
3,2010,1,2077388,355030,3501,355030,3501,327
4,2010,1,2077701,355030,3501,355030,3501,318
5,2010,1,2077388,355030,3501,355030,3501,303
6,2010,1,2077388,355030,3501,355030,3501,297
7,2010,1,2077701,355030,3501,355030,3501,296
8,2010,1,2077701,355030,3501,355030,3501,295


### Atualizando variável ano (biênio)

Valor da variável `bienio`:

- `0 (False) = 2010/11`
- `1 (True) = 2018/19`

In [5]:
df_deslc[:, dt.update(ano = (f.ano == 2018) | (f.ano == 2019))]
df_deslc.names = {'ano': 'bienio'}
df_deslc

,bienio,parto_normal,hosp_cnes,destino,hosp_regiao_saude,origem,regiao_saude,count
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,0,1,2077388,355030,3501,355030,3501,362
1,0,1,2077388,355030,3501,355030,3501,333
2,0,1,2077388,355030,3501,355030,3501,331
3,0,1,2077388,355030,3501,355030,3501,327
4,0,1,2077701,355030,3501,355030,3501,318
5,0,1,2077388,355030,3501,355030,3501,303
6,0,1,2077388,355030,3501,355030,3501,297
7,0,1,2077701,355030,3501,355030,3501,296
8,0,1,2077701,355030,3501,355030,3501,295


### Tabela agrupada

In [6]:
df_deslc = group_cols(df_deslc, df_deslc.names[:-1])
df_deslc

,bienio,parto_normal,hosp_cnes,destino,hosp_regiao_saude,origem,regiao_saude,count
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,0,27,260290,2601,260290,2601,1
1,0,0,27,260290,2601,260720,2601,1
2,0,0,94,260290,2601,260140,2603,2
3,0,0,94,260290,2601,260290,2601,58
4,0,0,94,260290,2601,260345,2601,3
5,0,0,94,260290,2601,260400,2602,1
6,0,0,94,260290,2601,260520,2603,5
7,0,0,94,260290,2601,260540,2602,2
8,0,0,94,260290,2601,260640,2604,2


- with all: `3_355_691` rows
- without age: `461_276` rows
- without age and cnes: `98_055` rows

## Tabela Municípios

### Colunas da tabela

In [7]:
cols_mun = {
  'cod_municipio': 'origem',
  'regiao': 'regiao',
  'uf': 'uf',
  # 'nome_uf': 'nome_uf',
  # 'macroregiao_saude': 'macroregiao_saude',
  # 'regiao_saude': 'regiao_saude',
  'nome_regiao_saude': 'nome_regiao_saude',
  # 'microregiao_saude': 'microregiao_saude',
  'municipio': 'municipio',
  'capital': 'capital',
  'latitude': 'latitude',
  'longitude': 'longitude',
  'socioecon': 'socioecon',
  ...: None
}

In [8]:
cols_mun_hosp = {
  'cod_municipio': 'destino',
  'uf': 'hosp_uf',
  'latitude': 'hosp_latitude',
  'longitude': 'hosp_longitude',
  ...: None
}

### Carregando tabela

In [9]:
path_mun = get_path('GESTANTES', 'municipios.csv.gzip')
df_mun = dt.fread(path_mun, columns=cols_mun)
df_mun

,origem,regiao,uf,municipio,capital,nome_regiao_saude,latitude,longitude,socioecon
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,355030,Sudeste,SP,São Paulo,1,Grande São Paulo,−23.548,−46.636,5
1,330455,Sudeste,RJ,Rio de Janeiro,1,Metropolitana I,−22.903,−43.208,5
2,130260,Norte,AM,Manaus,1,Entorno de Manaus,−3.102,−60.025,4
3,230440,Nordeste,CE,Fortaleza,1,Fortaleza,−3.717,−38.543,5
4,292740,Nordeste,BA,Salvador,1,Salvador,−12.971,−38.511,5
5,150140,Norte,PA,Belém,1,Metropolitana,−1.456,−48.504,4
6,310620,Sudeste,MG,Belo Horizonte,1,Belo Horizonte,−19.817,−43.956,1
7,431490,Sul,RS,Porto Alegre,1,CRS 01 Porto Alegre,−30.033,−51.23,5
8,410690,Sul,PR,Curitiba,1,Metropolitana,−25.428,−49.273,5


In [10]:
df_mun_hosp = dt.fread(path_mun, columns=cols_mun_hosp)
df_mun_hosp

,destino,hosp_uf,hosp_latitude,hosp_longitude
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,355030,SP,−23.548,−46.636
1,330455,RJ,−22.903,−43.208
2,130260,AM,−3.102,−60.025
3,230440,CE,−3.717,−38.543
4,292740,BA,−12.971,−38.511
5,150140,PA,−1.456,−48.504
6,310620,MG,−19.817,−43.956
7,431490,RS,−30.033,−51.23
8,410690,PR,−25.428,−49.273


### Abreviando valores coluna Região

In [11]:
regioes = {
  'Norte': 'NT',
  'Nordeste': 'ND',
  'Centro-Oeste': 'CO',
  'Sudeste': 'SD',
  'Sul': 'SL',
}

In [12]:
rr = df_mun[:, 'regiao']
rr.replace(regioes)
df_mun['regiao'] = rr
df_mun

,origem,regiao,uf,municipio,capital,nome_regiao_saude,latitude,longitude,socioecon
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,355030,SD,SP,São Paulo,1,Grande São Paulo,−23.548,−46.636,5
1,330455,SD,RJ,Rio de Janeiro,1,Metropolitana I,−22.903,−43.208,5
2,130260,NT,AM,Manaus,1,Entorno de Manaus,−3.102,−60.025,4
3,230440,ND,CE,Fortaleza,1,Fortaleza,−3.717,−38.543,5
4,292740,ND,BA,Salvador,1,Salvador,−12.971,−38.511,5
5,150140,NT,PA,Belém,1,Metropolitana,−1.456,−48.504,4
6,310620,SD,MG,Belo Horizonte,1,Belo Horizonte,−19.817,−43.956,1
7,431490,SL,RS,Porto Alegre,1,CRS 01 Porto Alegre,−30.033,−51.23,5
8,410690,SL,PR,Curitiba,1,Metropolitana,−25.428,−49.273,5


## Tabela Distâncias

### Carregando tabela

In [13]:
output_dist = get_path('GESTANTES', 'distancias.csv.gzip')
df_dist = dt.fread(output_dist)
df_dist = df_dist[:, df_dist.names[:-1]]
df_dist

,origem,destino,distancia,tempo
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,355030,355030,0,0
1,330455,330455,0,0
2,230440,230440,0,0
3,130260,130260,0,0
4,292740,292740,0,0
5,150140,150140,0,0
6,520870,520870,0,0
7,310620,310620,0,0
8,431490,431490,0,0


# Gerando tabela

## Juntando com outras

### Juntando com Distâncias

In [14]:
df_dist.key = ['origem', 'destino']
df_deslc = df_deslc[:, :, dt.join(df_dist)]
df_dist

origem,destino,distancia,tempo
▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
110001,110001,0,0
110001,110002,309.05,6.16906
110001,110004,81.2011,1.91775
110001,110006,353.417,5.53989
110001,110009,145.678,2.346
110001,110012,130.77,2.54781
110001,110013,357.511,7.13067
110001,110015,172.926,3.18425
110001,110018,114.061,1.87139


### Juntando com Municípios (res)

In [15]:
df_mun.key = 'origem'
df_deslc = df_deslc[:, :, dt.join(df_mun)]
df_mun

origem,regiao,uf,municipio,capital,nome_regiao_saude,latitude,longitude,socioecon
▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
110001,NT,RO,Alta Floresta D'Oeste,0,II Delegacia Regional de Saúde,−11.929,−61.996,1
110002,NT,RO,Ariquemes,0,Departamento de Epidemiologia,−9.913,−63.041,1
110003,NT,RO,Cabixi,0,III Delegacia Regional de Saúde,−13.492,−60.545,2
110004,NT,RO,Cacoal,0,II Delegacia Regional de Saúde,−11.438,−61.448,3
110005,NT,RO,Cerejeiras,0,III Delegacia Regional de Saúde,−13.189,−60.812,2
110006,NT,RO,Colorado do Oeste,0,III Delegacia Regional de Saúde,−13.117,−60.542,2
110007,NT,RO,Corumbiara,0,III Delegacia Regional de Saúde,−12.962,−60.887,2
110008,NT,RO,Costa Marques,0,I Delegacia Regional de Saúde,−12.445,−64.227,1
110009,NT,RO,Espigão D'Oeste,0,II Delegacia Regional de Saúde,−11.525,−61.013,3


### Juntando com Municípios (hosp)

In [16]:
df_mun_hosp.key = 'destino'
df_deslc = df_deslc[:, :, dt.join(df_mun_hosp)]
df_deslc

,bienio,parto_normal,hosp_cnes,destino,hosp_regiao_saude,origem,regiao_saude,count,distancia,tempo,…,longitude,socioecon,hosp_uf,hosp_latitude,hosp_longitude
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,0,27,260290,2601,260290,2601,1,0,0,…,−35.035,5,PE,−8.287,−35.035
1,0,0,27,260290,2601,260720,2601,1,16.8139,0.319111,…,−35.064,5,PE,−8.287,−35.035
2,0,0,94,260290,2601,260140,2603,2,73.9977,1.34642,…,−35.186,1,PE,−8.287,−35.035
3,0,0,94,260290,2601,260290,2601,58,0,0,…,−35.035,5,PE,−8.287,−35.035
4,0,0,94,260290,2601,260345,2601,3,42.4065,0.630306,…,−34.981,5,PE,−8.287,−35.035
5,0,0,94,260290,2601,260400,2602,1,81.5093,1.15006,…,−35.255,1,PE,−8.287,−35.035
6,0,0,94,260290,2601,260520,2603,5,27.9907,0.378972,…,−35.224,1,PE,−8.287,−35.035
7,0,0,94,260290,2601,260540,2602,2,103.117,1.53569,…,−35.389,1,PE,−8.287,−35.035
8,0,0,94,260290,2601,260640,2604,2,104,1.41492,…,−35.565,1,PE,−8.287,−35.035


## Criando colunas

In [17]:
df_deslc['mun_diff'] = f.origem != f.destino
df_deslc['regsau_diff'] = f.regiao_saude != f.hosp_regiao_saude
df_deslc

,bienio,parto_normal,hosp_cnes,destino,hosp_regiao_saude,origem,regiao_saude,count,distancia,tempo,…,hosp_uf,hosp_latitude,hosp_longitude,mun_diff,regsau_diff
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪,▪
0,0,0,27,260290,2601,260290,2601,1,0,0,…,PE,−8.287,−35.035,0,0
1,0,0,27,260290,2601,260720,2601,1,16.8139,0.319111,…,PE,−8.287,−35.035,1,0
2,0,0,94,260290,2601,260140,2603,2,73.9977,1.34642,…,PE,−8.287,−35.035,1,1
3,0,0,94,260290,2601,260290,2601,58,0,0,…,PE,−8.287,−35.035,0,0
4,0,0,94,260290,2601,260345,2601,3,42.4065,0.630306,…,PE,−8.287,−35.035,1,0
5,0,0,94,260290,2601,260400,2602,1,81.5093,1.15006,…,PE,−8.287,−35.035,1,1
6,0,0,94,260290,2601,260520,2603,5,27.9907,0.378972,…,PE,−8.287,−35.035,1,1
7,0,0,94,260290,2601,260540,2602,2,103.117,1.53569,…,PE,−8.287,−35.035,1,1
8,0,0,94,260290,2601,260640,2604,2,104,1.41492,…,PE,−8.287,−35.035,1,1


## Removendo valores NA

In [18]:
df_deslc = df_deslc[(
  f.latitude != None) & (
  f.longitude != None) & (
  f.hosp_latitude != None) & (
  f.hosp_longitude != None), :]
df_deslc

,bienio,parto_normal,hosp_cnes,destino,hosp_regiao_saude,origem,regiao_saude,count,distancia,tempo,…,hosp_uf,hosp_latitude,hosp_longitude,mun_diff,regsau_diff
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪,▪
0,0,0,27,260290,2601,260290,2601,1,0,0,…,PE,−8.287,−35.035,0,0
1,0,0,27,260290,2601,260720,2601,1,16.8139,0.319111,…,PE,−8.287,−35.035,1,0
2,0,0,94,260290,2601,260140,2603,2,73.9977,1.34642,…,PE,−8.287,−35.035,1,1
3,0,0,94,260290,2601,260290,2601,58,0,0,…,PE,−8.287,−35.035,0,0
4,0,0,94,260290,2601,260345,2601,3,42.4065,0.630306,…,PE,−8.287,−35.035,1,0
5,0,0,94,260290,2601,260400,2602,1,81.5093,1.15006,…,PE,−8.287,−35.035,1,1
6,0,0,94,260290,2601,260520,2603,5,27.9907,0.378972,…,PE,−8.287,−35.035,1,1
7,0,0,94,260290,2601,260540,2602,2,103.117,1.53569,…,PE,−8.287,−35.035,1,1
8,0,0,94,260290,2601,260640,2604,2,104,1.41492,…,PE,−8.287,−35.035,1,1


## Ordenando

### Colunas

In [19]:
cols_order = [
  # procedimento:
  'bienio',
  'parto_normal',
  # deslocamento:
  'origem',
  'destino',
  'mun_diff',
  'regsau_diff',
  'distancia',
  'tempo',
  # localização de residência:
  'regiao',
  'uf',
  # 'nome_uf',
  # 'macroregiao_saude',
  'regiao_saude',
  'nome_regiao_saude',
  # 'microregiao_saude',
  # 'municipio',
  'capital',
  'socioecon',
  'latitude',
  'longitude',
  # localização do hospital:
  'hosp_latitude',
  'hosp_longitude',
  'hosp_cnes',
  'hosp_regiao_saude',
  'hosp_uf',
  # contagem:
  'count',
]

In [20]:
df_deslc = df_deslc[:, cols_order]
df_deslc

,bienio,parto_normal,origem,destino,mun_diff,regsau_diff,distancia,tempo,regiao,uf,…,hosp_longitude,hosp_cnes,hosp_regiao_saude,hosp_uf,count
,▪,▪,▪▪▪▪,▪▪▪▪,▪,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,0,260290,260290,0,0,0,0,ND,PE,…,−35.035,27,2601,PE,1
1,0,0,260720,260290,1,0,16.8139,0.319111,ND,PE,…,−35.035,27,2601,PE,1
2,0,0,260140,260290,1,1,73.9977,1.34642,ND,PE,…,−35.035,94,2601,PE,2
3,0,0,260290,260290,0,0,0,0,ND,PE,…,−35.035,94,2601,PE,58
4,0,0,260345,260290,1,0,42.4065,0.630306,ND,PE,…,−35.035,94,2601,PE,3
5,0,0,260400,260290,1,1,81.5093,1.15006,ND,PE,…,−35.035,94,2601,PE,1
6,0,0,260520,260290,1,1,27.9907,0.378972,ND,PE,…,−35.035,94,2601,PE,5
7,0,0,260540,260290,1,1,103.117,1.53569,ND,PE,…,−35.035,94,2601,PE,2
8,0,0,260640,260290,1,1,104,1.41492,ND,PE,…,−35.035,94,2601,PE,2


### Linhas

In [21]:
df_deslc = df_deslc.sort(f.bienio, f.parto_normal, f.origem, f.destino)
df_deslc

,bienio,parto_normal,origem,destino,mun_diff,regsau_diff,distancia,tempo,regiao,uf,…,hosp_longitude,hosp_cnes,hosp_regiao_saude,hosp_uf,count
,▪,▪,▪▪▪▪,▪▪▪▪,▪,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,0,110001,110001,0,0,0,0,NT,RO,…,−61.996,2679477,1102,RO,35
1,0,0,110001,110012,1,1,130.77,2.54781,NT,RO,…,−61.944,2495279,1101,RO,6
2,0,0,110001,110013,1,1,357.511,7.13067,NT,RO,…,−61.981,2808617,1104,RO,1
3,0,0,110001,110018,1,0,114.061,1.87139,NT,RO,…,−61.193,2496534,1102,RO,1
4,0,0,110001,110020,1,1,505.856,9.16514,NT,RO,…,−63.904,4001303,1104,RO,2
5,0,0,110001,110028,1,0,48.6099,0.835667,NT,RO,…,−61.778,2495228,1102,RO,9
6,0,0,110001,351110,1,1,2196.35,30.8736,NT,RO,…,−48.973,2089327,3522,SP,1
7,0,0,110002,110002,0,0,0,0,NT,RO,…,−63.041,2494299,1104,RO,59
8,0,0,110002,110011,1,1,95.4727,2.33883,NT,RO,…,−62.466,2808609,1101,RO,2


# Exportando tabela

In [22]:
output_deslc = get_path('GESTANTES', 'deslocamentos.csv.gzip')
df_deslc.to_csv(output_deslc)

In [23]:
dt.fread(output_deslc)

,bienio,parto_normal,origem,destino,mun_diff,regsau_diff,distancia,tempo,regiao,uf,…,hosp_longitude,hosp_cnes,hosp_regiao_saude,hosp_uf,count
,▪,▪,▪▪▪▪,▪▪▪▪,▪,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,0,0,110001,110001,0,0,0,0,NT,RO,…,−61.996,2679477,1102,RO,35
1,0,0,110001,110012,1,1,130.77,2.54781,NT,RO,…,−61.944,2495279,1101,RO,6
2,0,0,110001,110013,1,1,357.511,7.13067,NT,RO,…,−61.981,2808617,1104,RO,1
3,0,0,110001,110018,1,0,114.061,1.87139,NT,RO,…,−61.193,2496534,1102,RO,1
4,0,0,110001,110020,1,1,505.856,9.16514,NT,RO,…,−63.904,4001303,1104,RO,2
5,0,0,110001,110028,1,0,48.6099,0.835667,NT,RO,…,−61.778,2495228,1102,RO,9
6,0,0,110001,351110,1,1,2196.35,30.8736,NT,RO,…,−48.973,2089327,3522,SP,1
7,0,0,110002,110002,0,0,0,0,NT,RO,…,−63.041,2494299,1104,RO,59
8,0,0,110002,110011,1,1,95.4727,2.33883,NT,RO,…,−62.466,2808609,1101,RO,2


# Tabelas Filtradas

## Tabela referência

In [24]:
df_deslc = dt.fread(output_deslc)

## Brasil (sem CNES)

In [25]:
df_deslc_brasil = df_deslc[:, f[:].remove(f.hosp_cnes)]
df_deslc_brasil = df_deslc_brasil[:, dt.sum(f.count), dt.by(df_deslc_brasil.names[:-1])]
df_deslc_brasil

,bienio,parto_normal,origem,destino,mun_diff,regsau_diff,distancia,tempo,regiao,uf,…,hosp_latitude,hosp_longitude,hosp_regiao_saude,hosp_uf,count
,▪,▪,▪▪▪▪,▪▪▪▪,▪,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,0,110001,110001,0,0,0,0,NT,RO,…,−11.929,−61.996,1102,RO,35
1,0,0,110001,110012,1,1,130.77,2.54781,NT,RO,…,−10.882,−61.944,1101,RO,6
2,0,0,110001,110013,1,1,357.511,7.13067,NT,RO,…,−9.444,−61.981,1104,RO,1
3,0,0,110001,110018,1,0,114.061,1.87139,NT,RO,…,−11.673,−61.193,1102,RO,1
4,0,0,110001,110020,1,1,505.856,9.16514,NT,RO,…,−8.762,−63.904,1104,RO,2
5,0,0,110001,110028,1,0,48.6099,0.835667,NT,RO,…,−11.733,−61.778,1102,RO,9
6,0,0,110001,351110,1,1,2196.35,30.8736,NT,RO,…,−21.138,−48.973,3522,SP,1
7,0,0,110002,110002,0,0,0,0,NT,RO,…,−9.913,−63.041,1104,RO,59
8,0,0,110002,110011,1,1,95.4727,2.33883,NT,RO,…,−10.439,−62.466,1101,RO,2


In [26]:
output_deslc_brasil = get_path('GESTANTES', 'deslocamentos_brasil.xlsx')
df_deslc_brasil.to_pandas().to_excel(output_deslc_brasil, index=False)

## RJ

### Com CNES

In [32]:
df_deslc_rj_cnes = df_deslc[(f.uf == 'RJ'), :]
df_deslc_rj_cnes

,bienio,parto_normal,origem,destino,mun_diff,regsau_diff,distancia,tempo,regiao,uf,…,hosp_longitude,hosp_cnes,hosp_regiao_saude,hosp_uf,count
,▪,▪,▪▪▪▪,▪▪▪▪,▪,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,0,0,330010,320245,1,1,614.613,7.66281,SD,RJ,…,−41.511,2448203,3202,ES,2
1,0,0,330010,330010,0,0,0,0,SD,RJ,…,−44.318,2280868,3301,RJ,57
2,0,0,330010,330010,0,0,0,0,SD,RJ,…,−44.318,2281384,3301,RJ,48
3,0,0,330010,351840,1,1,222.988,3.47778,SD,RJ,…,−45.193,2081512,3524,SP,1
4,0,0,330020,330020,0,0,0,0,SD,RJ,…,−42.343,2273098,3302,RJ,58
5,0,0,330020,330070,1,0,43.1787,0.650056,SD,RJ,…,−42.019,5903394,3302,RJ,3
6,0,0,330020,330455,1,1,118.052,1.55372,SD,RJ,…,−43.208,2270390,3305,RJ,1
7,0,0,330020,330455,1,1,118.052,1.55372,SD,RJ,…,−43.208,2273438,3305,RJ,1
8,0,0,330020,330520,1,0,31.0431,0.417139,SD,RJ,…,−42.103,2280477,3302,RJ,2


In [33]:
output_rj_cnes = get_path('GESTANTES', 'deslocamentos_rj_cnes.xlsx')
df_deslc_rj_cnes.to_csv(output_rj_cnes)
df_deslc_rj_cnes.to_pandas().to_excel(output_rj_cnes, index=False)

### Sem CNES

In [36]:
df_deslc_rj = df_deslc_rj_cnes[:, f[:].remove(f.hosp_cnes)]
df_deslc_rj = df_deslc_rj[:, dt.sum(f.count), dt.by(df_deslc_rj.names[:-1])]
df_deslc_rj

,bienio,parto_normal,origem,destino,mun_diff,regsau_diff,distancia,tempo,regiao,uf,…,hosp_latitude,hosp_longitude,hosp_regiao_saude,hosp_uf,count
,▪,▪,▪▪▪▪,▪▪▪▪,▪,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,0,330010,320245,1,1,614.613,7.66281,SD,RJ,…,−20.234,−41.511,3202,ES,2
1,0,0,330010,330010,0,0,0,0,SD,RJ,…,−23.007,−44.318,3301,RJ,105
2,0,0,330010,351840,1,1,222.988,3.47778,SD,RJ,…,−22.816,−45.193,3524,SP,1
3,0,0,330020,330020,0,0,0,0,SD,RJ,…,−22.873,−42.343,3302,RJ,58
4,0,0,330020,330070,1,0,43.1787,0.650056,SD,RJ,…,−22.879,−42.019,3302,RJ,3
5,0,0,330020,330455,1,1,118.052,1.55372,SD,RJ,…,−22.903,−43.208,3305,RJ,2
6,0,0,330020,330520,1,0,31.0431,0.417139,SD,RJ,…,−22.839,−42.103,3302,RJ,2
7,0,0,330020,330550,1,0,22.8379,0.348,SD,RJ,…,−22.92,−42.51,3302,RJ,1
8,0,0,330022,330600,1,0,25.1655,0.377083,SD,RJ,…,−22.117,−43.209,3303,RJ,32


In [37]:
output_rj = get_path('GESTANTES', 'deslocamentos_rj.xlsx')
df_deslc_rj.to_pandas().to_excel(output_rj, index=False)